In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
from IPython.display import clear_output

# Function to simulate and visualize infection spread with isolation, vaccination, and different contagiousness levels
def simulate_infection_graph(population_size, initial_infected, R0, recovery_rate, contagious_factor_for_iso, contagious_factor_for_uniso, isolation_rate, vaccination_rate, vaccine_efficacy, days):
    G = nx.erdos_renyi_graph(population_size, 0.05)  # Creating a random graph

    # Initialize node states (susceptible by default)
    for node in G.nodes:
        G.nodes[node]['state'] = 'susceptible'
        # Assign a random isolated or unisolated status based on isolation_rate
        G.nodes[node]['isolated'] = np.random.rand() < isolation_rate
        # Assign vaccinated or unvaccinated status based on vaccination rate
        G.nodes[node]['vaccinated'] = np.random.rand() < vaccination_rate

    # Infect a few initial nodes
    infected_nodes = np.random.choice(list(G.nodes), initial_infected, replace=False)
    for node in infected_nodes:
        G.nodes[node]['state'] = 'infected'

    # Initialize layout for consistent node positions
    pos = nx.spring_layout(G)

    # Simulate infection spread over multiple days
    for day in range(days):
        new_infected = []
        for node in G.nodes:
            if G.nodes[node]['state'] == 'infected':
                # Check neighbors for infection spread
                for neighbor in G.neighbors(node):
                    if G.nodes[neighbor]['state'] == 'susceptible':
                        # Determine contagiousness based on isolation
                        if G.nodes[node]['isolated']:
                            contagiousness = np.exp(-contagious_factor_for_iso * (day - 5)**2)
                        else:
                            contagiousness = np.exp(-contagious_factor_for_uniso * (day - 5)**2)

                        # Vaccinated individuals have reduced infection probability based on vaccine efficacy
                        infection_probability = R0 * contagiousness / 10
                        if G.nodes[neighbor]['vaccinated']:
                            infection_probability *= (1 - vaccine_efficacy)

                        # Infect the neighbor with the adjusted probability
                        if np.random.rand() < infection_probability:
                            new_infected.append(neighbor)

        # Infect new nodes
        for node in new_infected:
            G.nodes[node]['state'] = 'infected'

        # Recover some infected nodes based on the recovery rate
        infected_nodes = [node for node in G.nodes if G.nodes[node]['state'] == 'infected']
        recover_nodes = np.random.choice(infected_nodes, int(len(infected_nodes) * recovery_rate), replace=False)
        for node in recover_nodes:
            G.nodes[node]['state'] = 'recovered'

        # Visualize the graph dynamically with updated node states
        plt.figure(figsize=(8, 8))
        node_colors = ['red' if G.nodes[node]['state'] == 'infected' else
                       'green' if G.nodes[node]['state'] == 'recovered' else
                       'blue' for node in G.nodes]
        nx.draw(G, pos, node_color=node_colors, with_labels=False, node_size=100)
        plt.title(f'Day {day+1}: Infection Spread')

        # Adding a legend manually
        plt.scatter([], [], c='red', label='Infected')
        plt.scatter([], [], c='blue', label='Susceptible')
        plt.scatter([], [], c='green', label='Recovered')
        plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc="upper right", fontsize='large')

        # Display and clear the output to show dynamic updates
        plt.show()
        clear_output(wait=True)

    # Final visualization after all steps
    plt.figure(figsize=(8, 8))
    node_colors = ['red' if G.nodes[node]['state'] == 'infected' else
                   'green' if G.nodes[node]['state'] == 'recovered' else
                   'blue' for node in G.nodes]
    nx.draw(G, pos, node_color=node_colors, with_labels=False, node_size=100)
    plt.title(f'Final Infection Spread after {days} Days')

    # Adding a legend manually
    plt.scatter([], [], c='red', label='Infected')
    plt.scatter([], [], c='blue', label='Susceptible')
    plt.scatter([], [], c='green', label='Recovered')
    plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc="upper right", fontsize='large')

    plt.show()

    # Short explanation
    print(f"Simulation complete after {days} days.")
    print("Explanation:")
    print("1. Red nodes: represent infected individuals who can spread the disease to their neighbors.")
    print("2. Blue nodes: represent susceptible individuals who have not yet been infected.")
    print("3. Green nodes: represent recovered individuals who can no longer spread or catch the disease.")
    print("\nThe graph simulates how an infection spreads through a population over time, with isolated and unisolated individuals.")
    print("Isolated individuals spread the disease less due to reduced contact, while unisolated individuals spread more.")

# Interactive sliders for the simulation parameters
interact(simulate_infection_graph,
         population_size=IntSlider(min=50, max=500, step=10, value=100, description='Population Size'),
         initial_infected=IntSlider(min=1, max=50, step=1, value=5, description='Initial Infected'),
         R0=FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='R0 (Infection Rate)'),
         recovery_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Recovery Rate'),
         contagious_factor_for_iso=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Contagious Factor (Isolated)'),
         contagious_factor_for_uniso=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.3, description='Contagious Factor (Unisolated)'),
         isolation_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description='Isolation Rate'),
         vaccination_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.7, description='Vaccination Rate'),
         vaccine_efficacy=FloatSlider(min=0.5, max=1.0, step=0.01, value=0.9, description='Vaccine Efficacy'),
         days=IntSlider(min=1, max=20, step=1, value=10, description='Days'));


interactive(children=(IntSlider(value=100, description='Population Size', max=500, min=50, step=10), IntSlider…